In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [2]:
from discontinuitypy.datasets import IDsDataset
from discontinuitypy.utils.basic import resample
from beforerr.polars import pl_norm

from sunpy.time import TimeRange
from datetime import timedelta
from space_analysis.utils.speasy import Variables

06-Feb-24 15:49:16 WARNING  06-Feb-24 15:49:16: UserWarning: Traceback (most recent call last):       ]8;id=707665;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=491710;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py#205\205]8;;\
                              File                                                                                 
                            "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-package              
                            s/pdpipe/__init__.py", line 85, in <module>                                            
                                from . import skintegrate                                                          
                              File                                                                                 
                            "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-package              
                            s/pdpipe/skintegrate.py", line 20, in <module>                                         
                                from sklearn.base import BaseEstimator                                             
                            ModuleNotFoundError: No module named 'sklearn'                                         
                                                                                                                   
                                                                                                                   

                   WARNING  06-Feb-24 15:49:16: UserWarning: pdpipe: Scikit-learn or skutil import    ]8;id=270165;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=799943;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py#205\205]8;;\
                            failed. Scikit-learn-dependent pipeline stages will not be loaded.                     
                                                                                                                   

                   WARNING  06-Feb-24 15:49:16: UserWarning: Traceback (most recent call last):       ]8;id=833483;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=934851;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py#205\205]8;;\
                              File                                                                                 
                            "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-package              
                            s/pdpipe/__init__.py", line 105, in <module>                                           
                                from . import nltk_stages                                                          
                              File                                                                                 
                            "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-package              
                            s/pdpipe/nltk_stages.py", line 19, in <module>                                         
                                import nltk                                                                        
                            ModuleNotFoundError: No module named 'nltk'                                            
                                                                                                                   
                                                                                                                   

                   WARNING  06-Feb-24 15:49:16: UserWarning: pdpipe: nltk import failed.              ]8;id=624778;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=322425;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py#205\205]8;;\
                            nltk-dependent  pipeline stages will not be loaded.                                    
                                                                                                                   

In [37]:
start = '2019-04-06T12:00'
start = '2019-04-07T01:00'
end = '2019-04-07T12:00'

earth_start = '2019-04-09'
earth_end = '2019-04-12'

tau = timedelta(seconds=10)
ts = timedelta(seconds=1/264)

In [4]:
psp_timerange = TimeRange(start, end)
timerange_earth = TimeRange(earth_start, earth_end)

In [5]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [6]:
psp_mag_data = Variables(
    dataset="PSP_FLD_L2_MAG_RTN",
    parameters=["psp_fld_l2_mag_RTN"],
    timerange=validate(psp_timerange),
).to_polars()

PSP plasma data

In [7]:
dataset = "PSP_SWP_SPI_SF00_L3_MOM"
parameters = ["DENS", "VEL_RTN_SUN", "TEMP", "SUN_DIST"]
vec_cols = ["Vx RTN", "Vy RTN", "Vz RTN"]

psp_plasma = (
    Variables(dataset=dataset, parameters=parameters, timerange=validate(psp_timerange))
    .to_polars()
    .with_columns(plasma_speed=pl_norm(vec_cols))
    .rename(
        {
            "Density": "plasma_density",
            "Temperature": "plasma_temperature",
        }
    )
)

psp_plasma_tau = psp_plasma.pipe(resample, every=tau)

In [36]:
psp_ids_dataset = (
    IDsDataset(
        mag_data=psp_mag_data,
        plasma_data=psp_plasma_tau,
        tau=tau,
        ts=ts,
        vec_cols=vec_cols,
    )
    .find_events()
    .update_candidates_with_plasma_data()
    .export("../data/psp_ids_dataset.arrow")
)

06-Feb-24 16:09:48 WARNING  06-Feb-24 16:09:48: UserWarning: Distributing <class                      ]8;id=167115;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=92925;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py#205\205]8;;\
                            'pandas.core.frame.DataFrame'> object. This may take some time.                        
                                                                                                                   

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 9:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [46]:
# Resampled dataset

ts = timedelta(seconds=1 / 11)

psp_mag_data_r = psp_mag_data.pipe(resample, every=ts)

In [35]:
psp_ids_dataset_r = (
    IDsDataset(
        mag_data=psp_mag_data_r,
        plasma_data=psp_plasma_tau,
        tau=tau,
        ts=ts,
        vec_cols=vec_cols,
    )
    .find_events()
    .update_candidates_with_plasma_data()
    .export("../data/psp_ids_dataset_r.arrow")
)

06-Feb-24 16:09:35 WARNING  06-Feb-24 16:09:35: UserWarning: Distributing <class                      ]8;id=810181;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=617257;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py#205\205]8;;\
                            'pandas.core.frame.DataFrame'> object. This may take some time.                        
                                                                                                                   

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 9:   0%           Elapsed time: 00:00, estimated remaining time: ?

## Wind dataset

In [51]:
ts = timedelta(seconds=1 / 11)
tau = timedelta(seconds=30)

wind_vec_cols = ["VX (GSM)", "VY (GSM)", "VZ (GSM)"]


wind_mag_data = Variables(
    dataset="WI_H2_MFI",
    parameters=["BGSM"],
    timerange=validate(timerange_earth),
).to_polars()

wind_plasma_data = (
    Variables(
        dataset="WI_K0_SWE",
        parameters=["V_GSM", "Np"],
        timerange=validate(timerange_earth),
    )
    .to_polars()
    .with_columns(plasma_speed=pl_norm(wind_vec_cols))
    .rename(
        {
            "Ion Np": "plasma_density",
        }
    )
)

wind_ids_dataset = (
    IDsDataset(
        mag_data=wind_mag_data,
        plasma_data=wind_plasma_data,
        tau=tau,
        ts=ts,
        vec_cols=wind_vec_cols,
    )
    .find_events()
    .update_candidates_with_plasma_data()
    .export("../data/wind_ids_dataset_r.arrow")
)

06-Feb-24 17:02:22 WARNING  06-Feb-24 17:02:22: UserWarning: Distributing <class                      ]8;id=35254;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=385671;file:///Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/astropy/logger.py#205\205]8;;\
                            'pandas.core.frame.DataFrame'> object. This may take some time.                        
                                                                                                                   

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 36:   0%           Elapsed time: 00:00, estimated remaining time: ?